# **Tiempo en el mercado**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "publicacion" in nombre and "puebla" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_publicacion_puebla']

In [2]:
df_publicacion_puebla.rename(columns={'Tiempo_publicacion':'tiempo_de_publicacion'}, inplace=True)

# 1. Tiempo en el mercado

In [3]:
df_mercado_pachuca = df_publicacion_puebla[['id','tiempo_de_publicacion']]
df_mercado_pachuca

,id,tiempo_de_publicacion
0,1,1
1,2,8
2,3,5
3,4,16
4,5,16
...,...,...
4418,4433,153
4419,4434,45
4420,4435,118
4421,4436,88


In [4]:
print(df_mercado_pachuca.shape)
tiempo = df_mercado_pachuca['tiempo_de_publicacion'].unique()
print(tiempo)

(4423, 2)
[  1   8   5  16   2   7   4   3   6  12  14  96  19  45 365  17  25 189
 122 249  11  76 115  53  60  30  23 119 259  97 212 232 190 121  38  98
  27  58  59  95 105  79 235 183 288 206 229  81 120  40  31 290  54 323
  87 203 136  15  36  80  99  10 360 262 164  90 127  42 150  29  74 333
 176  44  18 140  48 145  43  64 172  35 118 114 211  21  88 198  86 160
 174  91  37 108  34 158 204  78 134  63  22  51   9 318  92  57 245 163
 241 329 170  73 149 269  39  32  26 137 168  71 248 130  93 123  56 113
  70  52  28  33 352 128  50  49 207 125 197 358 247 233  68  46 109 193
  94 126 268 173 110 152 102 129  72 165 180  83 221 162  24  13 191 100
 186 184  62  75 101 138 222   0 124 201  55 215 343 256 192 240 289 311
  85  69 218 133 141 219  20 135 194 302 171 298 199 261 317  61 304  77
 154  41 157 305 266 116 322 338 244  84  65 350 196 252 142  66 205 117
  47 132 147 112 156 260 144 143 253 185 225 223 330 308 297 270 310 146
  67 332 255 327 178 169 167 131 283 281 

## Asignar rangos

In [5]:
# Definir la función de categorización
def categorizar(valor):
    if isinstance(valor, (int, float)):  # Verificar si el valor es un número (int o float)
        if 0 <= valor <= 10:
            return '0-10'
        elif 11 <= valor <= 30:
            return '11-30'
        elif 31 <= valor <= 100:
            return '31-100'
        elif 101 <= valor <= 364:
            return '101-364'
        else:
            return '>=365'
    elif isinstance(valor, str) and valor.isdigit():  # Verificar si el valor es una cadena numérica
        valor = int(valor)  # Convertir a entero si es un número
        if 0 <= valor <= 10:
            return '0-10'
        elif 11 <= valor <= 30:
            return '11-30'
        elif 31 <= valor <= 100:
            return '31-100'
        elif 101 <= valor <= 364:
            return '101-364'
        else:
            return '>=365'
    else:
        return 'No numérico'  # Manejar valores no numéricos

# Lista de nombres de los DataFrames
dataframes_name = ['df_mercado_pachuca']

# Lista de DataFrames
dataframes_list = [df_mercado_pachuca]

# Definir el orden de las categorías
todas_categorias = ['0-10', '11-30', '31-100', '101-364', '>=365']

# Diccionario para almacenar los DataFrames de conteo por nombre de DataFrame original
dataframes_conteo_por_nombre = {}

# Iterar sobre cada DataFrame
for name, df in zip(dataframes_name, dataframes_list):
    # Aplicar la función de categorización
    df = df.copy()
    df['dias'] = df['tiempo_de_publicacion'].apply(categorizar)
    # Contar el número de registros únicos por cada valor único en la columna 'dias'
    conteo_por_dias = df['dias'].value_counts().reindex(todas_categorias, fill_value=0)
    # Crear un nuevo DataFrame de conteo
    df_conteo = pd.DataFrame({'dias': conteo_por_dias.index, 'conteo': conteo_por_dias.values})
    # Ordenar el DataFrame según el orden definido en todas_categorias
    df_conteo['dias'] = pd.Categorical(df_conteo['dias'], categories=todas_categorias, ordered=True)
    df_conteo = df_conteo.sort_values(by='dias')
    # Almacenar el DataFrame de conteo en el diccionario por nombre de DataFrame original
    dataframes_conteo_por_nombre[name] = df_conteo
    # Imprimir el nombre del DataFrame de conteo y su contenido
    nuevo_nombre_df_conteo = 'conteo_' + name
    globals()[nuevo_nombre_df_conteo] = df_conteo
    print(f'Nombre del DataFrame de conteo: {nuevo_nombre_df_conteo}')
    print(df_conteo)

Nombre del DataFrame de conteo: conteo_df_mercado_pachuca
      dias  conteo
0     0-10    1680
1    11-30     762
2   31-100     703
3  101-364     851
4    >=365     427


In [6]:
conteo_df_mercado_pachuca

,dias,conteo
0,0-10,1680
1,11-30,762
2,31-100,703
3,101-364,851
4,>=365,427


In [7]:
values = conteo_df_mercado_pachuca['conteo']
labels = conteo_df_mercado_pachuca['dias']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=labels, 
    y=values, 
    marker_color=colores,  # Especifica los colores de las barras
    text=values,  # Texto que se mostrará en las barras 
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    #title='Número de unidades por días en el mercado',  # Título de la gráfica
    xaxis=dict(title='Rango días'),  # Título del eje x
    yaxis=dict(
        title='Propiedades', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
             ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)    
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_tiempo_mercado', carpeta='assets\graficas')
fig.show()

____
# 2. Riesgo

In [8]:
# Definir los valores para la columna 'stock'
stock_values = []
for dias in conteo_df_mercado_pachuca['dias']:
    if dias == '>=365':
        stock_values.append('Más de un año en stock')
    else:
        stock_values.append('Menos de un año en stock')

# Crear el DataFrame 'riesgo'
riesgo = pd.DataFrame({
    'stock': stock_values,
    'casas': conteo_df_mercado_pachuca['conteo']
})

# Agrupar por la columna 'stock' y sumar los valores de 'casas'
riesgo_stock = riesgo.groupby('stock').sum().reset_index()
# Ordenar el DataFrame para que 'Menos de un año en stock' quede abajo
riesgo_stock = riesgo_stock.sort_values(by='stock', ascending=False)
riesgo_stock

,stock,casas
1,Más de un año en stock,427
0,Menos de un año en stock,3996


In [9]:
labels = riesgo_stock['stock']
values = riesgo_stock['casas']
colores = ['#9500ff', '#ff0059']
# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores,)
)])
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10),)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_pie_riesgo', carpeta='assets\graficas')
fig.show()